# HU 4: Conexión Power BI con PostgreSQL

In [1]:
from sqlalchemy import create_engine
import pandas as pd
from urllib.parse import quote_plus

usuario = "postgres"
password = quote_plus("P@ssw0rd1234")
host = "localhost"
puerto = 5432
base_datos = "ventas_db"

url = f"postgresql+psycopg2://{usuario}:{password}@{host}:{puerto}/{base_datos}"
engine = create_engine(url)


In [2]:
df_limpio = pd.read_csv("ventas_limpio.csv")
df_limpio.head()


,Ciudad,Fecha,Producto,Tipo_Producto,Cantidad,Precio_Unitario,Tipo_Venta,Tipo_Cliente,Descuento,Costo_Envio,...,Fecha_era_nulo,Producto_era_nulo,Tipo_Producto_era_nulo,Cantidad_era_nulo,Precio_Unitario_era_nulo,Tipo_Venta_era_nulo,Tipo_Cliente_era_nulo,Descuento_era_nulo,Costo_Envio_era_nulo,Total_Pagado_era_nulo
0,Santiago,2025-10-30,Arepa,Abarrotes,2.0,3681.0,Online,Minorista,0.20,0.0,...,0,0,0,0,0,0,0,0,0,0
1,Cordoba,2025-11-17,Arepa,Abarrotes,7.0,2321.0,Distribuidor,Gobierno,0.15,0.0,...,0,0,0,0,0,0,0,0,0,0
2,Barranquilla,2025-10-22,Leche,Lacteo,9.0,3540.0,Distribuidor,Gobierno,0.20,0.0,...,0,0,0,0,0,0,0,0,0,0
3,New York,2025-10-20,Cereal,Abarrotes,3.0,3287.0,Tiendafisica,Gobierno,0.05,0.0,...,0,0,0,0,0,0,0,0,0,0
4,Madrid,2025-10-20,Leche,Lacteo,2.0,3414.0,Distribuidor,Mayorista,0.00,0.0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
with engine.connect() as conn:
    print(pd.read_sql("SELECT NOW()", conn))


                               now
0 2025-11-27 19:56:43.383091+00:00


In [5]:
df_limpio.to_sql(
    "ventas_limpia",
    con=engine,
    if_exists="replace",  # reemplaza si ya existe
    index=False
)


898

## 1. Configuración de la conexión

- Herramienta: **Power BI Desktop**  
- Origen de datos: **PostgreSQL**  
- Servidor: `localhost`  
- Base de datos: `ventas_db`  
- Modo de conexión: **DirectQuery** (conexión directa)  
- Tabla principal importada: `ventas`  

La conexión se realizó desde **Inicio → Obtener datos → PostgreSQL database**, ingresando las credenciales del usuario de PostgreSQL y seleccionando la tabla de ventas.

---

## 2. Modelo estrella implementado

Se diseñó un modelo de datos con la siguiente estructura:

- **Tabla de hechos**
  - `Fact_Ventas`  
  - Columnas clave: `Fecha`, `Producto`, `Ciudad`, `Tipo_Cliente`, `Tipo_Venta`, `Cantidad`, `Total_Pagado`, etc.

- **Tablas de dimensión**
  - `Dim_Producto` (Producto, Tipo_Producto)
  - `Dim_Ciudad` (Ciudad)
  - `Dim_Tiempo` (Fecha, Año, Mes, Día)
  - `Dim_Tipo_Cliente` (Tipo_Cliente)

Las dimensiones se generaron en Power Query a partir de la tabla de hechos mediante la opción **Referencia** y la eliminación de duplicados.

Las relaciones creadas fueron de tipo **1:* (uno a varios)** desde cada dimensión hacia `Fact_Ventas`, usando las columnas:

- `Dim_Producto[Producto]` → `Fact_Ventas[Producto]`
- `Dim_Ciudad[Ciudad]` → `Fact_Ventas[Ciudad]`
- `Dim_Tiempo[Fecha]` → `Fact_Ventas[Fecha]`
- `Dim_Tipo_Cliente[Tipo_Cliente]` → `Fact_Ventas[Tipo_Cliente]`

---

## 3. Validación de integridad y consistencia

Para asegurar que los datos en Power BI coinciden con PostgreSQL se realizaron las siguientes validaciones:

1. **Recuento de filas**

   - PostgreSQL: `SELECT COUNT(*) FROM ventas;`  
   - Power BI (medida DAX): `Total Filas Ventas = COUNTROWS(Fact_Ventas)`  

   Los resultados coinciden, confirmando que no hubo pérdida de registros en la conexión.

2. **Total de ventas**

   - PostgreSQL: `SELECT SUM("Total_Pagado") FROM ventas;`  
   - Power BI: `Total Ventas = SUM(Fact_Ventas[Total_Pagado])`  

   Ambos valores son iguales, validando la consistencia del campo económico principal.

3. **Prueba de relaciones**

   - Se crearon visualizaciones utilizando dimensiones (Producto, Ciudad, Fecha) y la medida `Total Ventas`.  
   - Al aplicar filtros por ciudad, producto y año, las ventas se ajustan correctamente, demostrando que las relaciones del modelo estrella funcionan como se espera.

---

## 4. Validación visual de integridad y consistencia — Power BI

La validación visual del modelo confirmó que todas las filas de la tabla de hechos se relacionan correctamente con las dimensiones correspondientes. Los filtros aplicados en los visuales (por ciudad, producto y fecha) afectan correctamente a las medidas, lo que garantiza relaciones activas y consistentes. No se identificaron valores en blanco en las dimensiones, lo que indica ausencia de datos huérfanos. Además, los totales calculados en Power BI coinciden con los obtenidos en Python y PostgreSQL, validando la integridad numérica del modelo. Finalmente, el comportamiento ordenado de la serie temporal demuestra que la Dim_Tiempo está correctamente construida y sin duplicados.
